In [2]:
from keras.layers import Dense, Dropout, LSTM, Embedding
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential

C:\Users\konya\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
import pandas as pd
import numpy as np

In [30]:
train_data = pd.read_csv("C:\\Users\\konya\\Desktop\\DNA TFBP\\DNA_TranscriptionFactorBindingPrediction\\data\\train.csv");
test_data = pd.read_csv("C:\\Users\\konya\\Desktop\\DNA TFBP\\DNA_TranscriptionFactorBindingPrediction\\data\\test.csv");
train_len = int(len(train_data))
print(train_len)

2000


In [31]:
#split train data into train and validation data
split_ratio = 0.8

train_size = int( train_len * (split_ratio))

#before taking partitions shuffle rows
#this is because all same class tuples are grouped together
train_data = train_data.sample(frac=1).reset_index(drop=False)

X_train = np.array(train_data['sequence'][0:train_size])
Y_train = np.array(train_data['label'][0:train_size])
X_test = np.array(train_data['sequence'][train_size:])
Y_test = np.array(train_data['label'][train_size:])


In [76]:
#preprocessing of DNA sequences
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(split='', char_level=True)
tokenizer.fit_on_texts(X_train)

print(tokenizer.word_counts)
print(tokenizer.document_count)
print(tokenizer.word_index)
print(tokenizer.word_docs)

encoded_X_train = tokenizer.texts_to_matrix(X_train, mode='tfidf') #ode = freq,count,binary
print(encoded_X_train[0])

OrderedDict([('C', 5555), ('A', 4816), ('G', 7521), ('T', 4508)])
1600
{'G': 1, 'C': 2, 'A': 3, 'T': 4}
{'T': 1362, 'A': 1426, 'G': 1525, 'C': 1522}
[0.         1.71122195 1.71362663 1.96229129 0.77651412]


In [85]:
#LSTM model in keras
def create_model(input_length):
    LSTM_model = Sequential()
    print(input_length)
    LSTM_model.add(Embedding(4,32, input_length = 5))
    LSTM_model.add(LSTM(output_dim=256, activation='sigmoid', recurrent_activation='hard_sigmoid', return_sequences=True))
    LSTM_model.add(Dropout(0.5))
    LSTM_model.add(LSTM(output_dim=256, activation='sigmoid', recurrent_activation='hard_sigmoid'))
    LSTM_model.add(Dropout(0.5))
    LSTM_model.add(Dense(1, activation='sigmoid'))


    LSTM_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    return LSTM_model

In [78]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(X_test.shape)

(1600,)
(1600,)
(400,)
(400,)


In [86]:
model = create_model(train_size)

hist = model.fit(encoded_X_train, Y_train, batch_size=50, epochs=10, validation_split = 0.2, verbose = 1)


1600


C:\Users\konya\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(activation="sigmoid", recurrent_activation="hard_sigmoid", return_sequences=True, units=256)`
  
C:\Users\konya\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(activation="sigmoid", recurrent_activation="hard_sigmoid", units=256)`
  


Train on 1280 samples, validate on 320 samples
Epoch 1/10
1280/1280 [==============================] - 6s 5ms/step - loss: 0.7429 - acc: 0.5008 - val_loss: 0.7524 - val_acc: 0.4750
Epoch 2/10
1280/1280 [==============================] - 2s 2ms/step - loss: 0.7196 - acc: 0.4930 - val_loss: 0.6833 - val_acc: 0.7469
Epoch 3/10
1280/1280 [==============================] - 2s 2ms/step - loss: 0.6979 - acc: 0.5422 - val_loss: 0.6564 - val_acc: 0.7969
Epoch 4/10
1280/1280 [==============================] - 2s 2ms/step - loss: 0.6482 - acc: 0.6148 - val_loss: 0.5643 - val_acc: 0.7469
Epoch 5/10
1280/1280 [==============================] - 2s 2ms/step - loss: 0.5397 - acc: 0.7344 - val_loss: 0.5452 - val_acc: 0.7625
Epoch 6/10
1280/1280 [==============================] - 2s 2ms/step - loss: 0.4668 - acc: 0.7914 - val_loss: 0.3961 - val_acc: 0.8312
Epoch 7/10
1280/1280 [==============================] - 2s 2ms/step - loss: 0.4082 - acc: 0.8078 - val_loss: 0.3649 - val_acc: 0.8531
Epoch 8/10
1280

In [88]:
encoded_X_test = tokenizer.texts_to_matrix(X_test, mode='tfidf')
score, acc = model.evaluate(encoded_X_test, Y_test, batch_size=1)
print('Test score:', score)
print('Test accuracy:', acc)

400/400 [==============================] - 2s 4ms/step
Test score: 0.45233742103679103
Test accuracy: 0.7925
